In [11]:
from datetime import datetime, date, timedelta
import tkinter.ttk as ttk
import tkinter as tk
from tkinter import messagebox
import pyodbc

# Create a connection to the SQL Server database
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-6Q5ES10;'
                      'Database=LibraryManagementSystem;'
                      'Trusted_Connection=yes;')
# Create a cursor object
cursor = conn.cursor()
   

#create login wimdow

root = tk.Tk()
root.title("Library Management System")
root.geometry("500x500") 
id_label = tk.Label(root, text="ID", font=("Calibri", 14, "bold"))
id_label.place(relx=0.5, rely=0.3, anchor="center")
id_entry = tk.Entry(root)
id_entry.place(relx=0.5, rely=0.34, anchor="center")
password_label = tk.Label(root, text="Password",font=("Calibri", 14, "bold"))
password_label.place(relx=0.5, rely=0.4, anchor="center")
password_entry = tk.Entry(root, show='*')
password_entry.place(relx=0.5, rely=0.44, anchor="center")

def login():
    # Get the ID and password from the entry fields
    username = id_entry.get()
    password = password_entry.get()

    # SQL query to fetch the username and password from the admin table
    cursor.execute("SELECT * FROM Admin WHERE name = ? AND password = ?", (username, password))

    # Check if the ID and password are correct
    if cursor.fetchone():
        root.destroy()
        open_dashboard(username, st_date, st_time)
       
    else:
        # Show an error message
        error_label = tk.Label(root, text="Invalid ID or password", font=("Calibri", 16, "bold"), fg="red")
        error_label.place(relx=0.5, rely=0.6, anchor="center")

def change():
    chg = tk.Tk()
    chg.title("Change Password")
    chg.geometry("400x400")
    un_label = tk.Label(chg, text="Username", font=("Calibri", 12, "bold"))
    un_label.place(relx=0.15,rely=0.10, anchor="w")
    un_entry=tk.Entry(chg, font=("Calibri", 12))
    un_entry.place(relx=0.5, rely=0.10, anchor="w")
    op_label = tk.Label(chg, text="Old Password", font=("Calibri", 12, "bold"))
    op_label.place(relx=0.15,rely=0.20, anchor="w")
    op_entry=tk.Entry(chg, font=("Calibri", 12))
    op_entry.place(relx=0.5, rely=0.20, anchor="w")
    np_label = tk.Label(chg, text="New Password", font=("Calibri", 12, "bold"))
    np_label.place(relx=0.15,rely=0.30, anchor="w")
    np_entry=tk.Entry(chg, font=("Calibri", 12))
    np_entry.place(relx=0.5, rely=0.30, anchor="w")
    

    def chnge():
        username = un_entry.get()
        old_password = op_entry.get()
        new_password = np_entry.get()
        cursor.execute("UPDATE Admin SET password = ? WHERE name = ? AND password = ?", (new_password, username, old_password))
        if cursor.rowcount > 0:
            conn.commit()
        else:
            # Show an error message
            error_label = tk.Label(chg, text="Wrong ID or password", font=("Calibri", 16, "bold"), fg="red")
            error_label.place(relx=0.5, rely=0.6, anchor="center")
        chg.destroy()
        un_entry.delete(0, tk.END)
        op_entry.delete(0, tk.END)
        np_entry.delete(0, tk.END)
        
    ch_button = tk.Button(chg, text="Change", command=chnge, font=("Calibri", 12, "bold"), bg="yellow")
    ch_button.place(relx=0.5, rely=0.42, anchor="e")        
    chg.mainloop()
    
login_button = tk.Button(root, text="Login", command=login, font=("Calibri", 16, "bold"), bg="yellow")
login_button.place(relx=0.5, rely=0.52, anchor="center")

change_button = tk.Button(root, text="Change Password", command=change, font=("Calibri", 12, "bold"), fg="blue")
change_button.place(relx=0.9, rely=0.08, anchor="e")
root.mainloop()

c_date=datetime.now()
st_date=c_date.strftime("%A, %d %B, %Y")
st_time=c_date.strftime("%I:%M %p")

#create dashboard window

def addin(isbn_entry, title_entry, author_entry, category_entry, publisher_entry, pubdate_entry, price_entry, quantity_entry):
    isbn = isbn_entry.get()
    title = title_entry.get()
    author = author_entry.get()
    category = category_entry.get()
    publisher = publisher_entry.get()
    p_date = pubdate_entry.get()
    fine = int(price_entry.get())
    quantity = int(quantity_entry.get())
    
    querry="""
    IF EXISTS (SELECT * FROM Books WHERE isbn = ?)
    BEGIN
        UPDATE Books
        SET total_quantity = total_quantity + ? 
        WHERE isbn = ? 

        UPDATE Books
        SET current_quantity = current_quantity + ?
        WHERE isbn = ? 

    END
    ELSE
    BEGIN
        INSERT INTO Books (isbn, title, author, category, publisher, publication_date, fine_price, total_quantity, current_quantity)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    END"""
    cursor.execute(querry, (isbn, quantity, isbn, quantity, isbn, isbn, title, author, category, publisher, p_date, fine, quantity, quantity))
    conn.commit()
    conn.commit()
    isbn_entry.delete(0, tk.END)
    title_entry.delete(0, tk.END)
    author_entry.delete(0, tk.END)
    category_entry.delete(0,tk.END)
    publisher_entry.delete(0, tk.END)
    pubdate_entry.delete(0, tk.END)
    price_entry.delete(0, tk.END)
    quantity_entry.delete(0, tk.END)
    quantity_entry.delete(0, tk.END)

def show_books():
    show = tk.Tk()
    show.title ("Show Books")
    
    columns = ('ISBN', 'Title', 'Author', 'Category', 'Publisher', 'Publication Date', 'Fine', 'Quantity')
    books = ttk.Treeview(show, columns=columns)

    books.heading('#0', text=' ')
    books.column('#0', width=0, stretch=tk.NO)
    for column in columns:
        books.heading(column, text=column)
        books.column(column, anchor=tk.W, width=150)
    
    books.pack(side='right', fill='both')
    populate_books(books, show)

def populate_books(books, show):
    # Get the list of books from the database
    cursor.execute('SELECT isbn, title, author, category, publisher, publication_date, fine_price, current_quantity FROM Books')
    rows = cursor.fetchall()
    
    # Clear the existing data in the books widget
    books.delete(*books.get_children())

    # Add the new data to the books widget
    for row in rows:
        values = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])
        books.insert('', 'end', values=values)
        
    show.mainloop()


def return_book():
    return_window = tk.Tk()
    return_window.title("Return Book")
    return_window.geometry("500x500")

    # Create a label and entry 
    id_label = tk.Label(return_window, text="Student ID", font=("Calibri", 14, "bold"))
    id_label.place(relx=0.5, rely=0.3, anchor="center")
    id_entry = tk.Entry(return_window, font=("Calibri", 12))
    id_entry.place(relx=0.5, rely=0.36, anchor="center")
    isbn_label = tk.Label(return_window, text="Book ISBN",font=("Calibri", 14, "bold"))
    isbn_label.place(relx=0.5, rely=0.46, anchor="center")
    isbn_entry = tk.Entry(return_window, font=("Calibri", 12))
    isbn_entry.place(relx=0.5, rely=0.52, anchor="center")
  
    def ret():
        return_date=datetime.now()
        r_date=return_date.strftime("%Y-%m-%d")
        book_isbn = isbn_entry.get()
        student_id = id_entry.get()

        # Look up the book by ISBN
        cursor.execute("SELECT fine_price FROM Books WHERE isbn = ?", (book_isbn))
        fine = cursor.fetchone()
        
        cursor.execute("SELECT due_date FROM IssuedBooks WHERE book_isbn = ? AND student_id = ? AND status = ?", (book_isbn, student_id, "pending"))
        row = cursor.fetchone()
        ddate = row.due_date
        due_date = datetime.strptime(ddate, "%Y-%m-%d")
        days_late = (return_date - due_date).days
        print(days_late)
        cursor.execute("UPDATE Books SET current_quantity = current_quantity + 1 WHERE isbn = ? ", (book_isbn))
        cursor.execute("UPDATE IssuedBooks SET return_date = ? where book_isbn = ? AND student_id = ? AND status = ? ", (r_date, book_isbn, student_id, "pending"))
        if days_late > 7:
            # The book is overdue
            print("Updating status to 'late returned'")
            cursor.execute("UPDATE IssuedBooks SET status = ? WHERE book_isbn = ? AND student_id = ? AND status = ?", ("late return", book_isbn, student_id, "pending"))
            cursor.execute("UPDATE IssuedBooks SET fine = ? WHERE book_isbn = ? AND student_id = ? AND status = ?", (fine[0], book_isbn, student_id, "pending"))
        else:
            # The book is not overdue
            print("Updating status to 'returned'")
            cursor.execute("UPDATE IssuedBooks SET status = ? WHERE book_isbn = ? AND student_id = ? AND status = ?", ("returned", book_isbn, student_id, "pending"))
        conn.commit()
        
        isbn_entry.delete(0, tk.END)
        id_entry.delete(0, tk.END)
        conn.close()                                                                                                                            
    r_button = tk.Button(return_window, text="Return", command = ret, font=("Calibri", 16, "bold"), bg="yellow")
    r_button.place(relx=0.5, rely=0.72, anchor="center")
    return_window.mainloop()

    

    

def issue_book():
    issue_window = tk.Tk()
    issue_window.title("Issue Book")
    issue_window.geometry("500x500")

    # Create a label and entry 
    id_label = tk.Label(issue_window, text="Student ID", font=("Calibri", 14, "bold"))
    id_label.place(relx=0.5, rely=0.3, anchor="center")
    id_entry = tk.Entry(issue_window, font=("Calibri", 12))
    id_entry.place(relx=0.5, rely=0.36, anchor="center")
    isbn_label = tk.Label(issue_window, text="Book ISBN",font=("Calibri", 14, "bold"))
    isbn_label.place(relx=0.5, rely=0.46, anchor="center")
    isbn_entry = tk.Entry(issue_window, font=("Calibri", 12))
    isbn_entry.place(relx=0.5, rely=0.52, anchor="center")

    def issue():
        curr_date=datetime.now()
        sev_date= datetime.now() + timedelta(days=7)
        c_date=curr_date.strftime("%Y-%m-%d")
        s_date=sev_date.strftime("%Y-%m-%d") 
        book_isbn = isbn_entry.get()
        student_id = id_entry.get()

        # Look up the book by ISBN
        cursor.execute("SELECT title FROM Books WHERE isbn = ?", (book_isbn))
        book = cursor.fetchone()
        if not book:
            tk.messagebox.showerror("Error", "Invalid book ISBN")
            return

        # Look up the student by ID
        cursor.execute("SELECT name FROM Students WHERE student_id = ?", (student_id))
        student = cursor.fetchone()
        if not student:
            tk.messagebox.showerror("Error", "Invalid student ID")
            return

        # Check if the student already has two books checked out
        cursor.execute("SELECT COUNT(*) FROM IssuedBooks WHERE student_id = ? AND status = ?", (student_id, "pending"))
        num_books = cursor.fetchone()[0]
        if num_books >= 3:
            tk.messagebox.showerror("Error", "Student already has three books checked out")
            return

        # Issue the book to the student
        cursor.execute("UPDATE Books SET current_quantity = current_quantity - 1 WHERE isbn = ?", (book_isbn))
        cursor.execute("INSERT INTO IssuedBooks (student_id, student_name, book_isbn, book_title, issue_date, due_date, status) VALUES (?, ?, ?, ?, ?, ?,?)", (student_id, student[0], book_isbn, book[0], c_date, s_date, "pending"))
        conn.commit()
        tk.messagebox.showinfo("Success", "Book issued successfully!")
        isbn_entry.delete(0, tk.END)
        id_entry.delete(0, tk.END)
    i_button = tk.Button(issue_window, text="Issue", command = issue, font=("Calibri", 16, "bold"), bg="yellow")
    i_button.place(relx=0.5, rely=0.72, anchor="center")
    issue_window.mainloop()

    
def delete_book():
    delete = tk.Tk()
    delete.title("Delete Book")
    delete.geometry("500x500")

    isbn_label = tk.Label(delete, text="Book ISBN", font=("Calibri", 14, "bold"))
    isbn_label.place(relx=0.5, rely=0.3, anchor="center")
    isbn_entry = tk.Entry(delete, font=("Calibri", 12))
    isbn_entry.place(relx=0.5, rely=0.36, anchor="center")
    
    quantity_label = tk.Label(delete, text="Quantity",font=("Calibri", 14, "bold"))
    quantity_label.place(relx=0.5, rely=0.46, anchor="center")
    quantity_entry = tk.Entry(delete, font=("Calibri", 12))
    quantity_entry.place(relx=0.5, rely=0.52, anchor="center")

    def delet():
      book_isbn = isbn_entry.get()
      quantity = quantity_entry.get()

      cursor.execute("SELECT total_quantity FROM Books WHERE isbn = ?", (book_isbn))
      tq = cursor.fetchone()
      total_quantity = tq.total_quantity
      if int(quantity) < int(total_quantity):
          cursor.execute("UPDATE Books SET total_quantity = total_quantity - ? WHERE isbn = ? ", (quantity, book_isbn))
          cursor.execute("UPDATE Books SET current_quantity = current_quantity - ? WHERE isbn = ? ", (quantity, book_isbn))
      elif int(quantity) == int(total_quantity):
          cursor.execute("DELETE * FROM Books WHERE isbn = ? ", (book_isbn))

      elif int(quantity) > int(total_quantity):
          tk.messagebox.showerror("Error", "Entered quantity is more than present quantity of Book")
          return
      conn.commit()
      isbn_entry.delete(0, tk.END)
      quantity_entry.delete(0, tk.END) 

        
    d_button = tk.Button(delete, text="Delete", command = delet, font=("Calibri", 16, "bold"), bg="yellow")
    d_button.place(relx=0.5, rely=0.72, anchor="center")
    delete.mainloop()
    

def add_book():
    add = tk.Tk()
    add.title("Add Book")
    add.geometry("600x1000")

    isbn_label = tk.Label(add, text="ISBN Number", font=("Calibri", 12, "bold"))
    isbn_label.place(relx=0.15,rely=0.10, anchor="w")
    isbn_entry=tk.Entry(add, font=("Calibri", 12))
    isbn_entry.place(relx=0.4, rely=0.10, anchor="w")
    title_label = tk.Label(add, text="Book Title", font=("Calibri", 12, "bold"))
    title_label.place(relx=0.15,rely=0.15, anchor="w")
    title_entry=tk.Entry(add, font=("Calibri", 12))
    title_entry.place(relx=0.4, rely=0.15, anchor="w")
    author_label = tk.Label(add, text="Book Author", font=("Calibri", 12, "bold"))
    author_label.place(relx=0.15,rely=0.20, anchor="w")
    author_entry=tk.Entry(add, font=("Calibri", 12))
    author_entry.place(relx=0.4, rely=0.20, anchor="w")
    category_label = tk.Label(add, text="Category", font=("Calibri", 12, "bold"))
    category_label.place(relx=0.15,rely=0.25, anchor="w")
    category_entry=tk.Entry(add, font=("Calibri", 12))
    category_entry.place(relx=0.4, rely=0.25, anchor="w")
    publisher_label = tk.Label(add, text="Publisher", font=("Calibri", 12, "bold"))
    publisher_label.place(relx=0.15,rely=0.30, anchor="w")
    publisher_entry=tk.Entry(add, font=("Calibri", 12))
    publisher_entry.place(relx=0.4, rely=0.30, anchor="w")
    pubdate_label = tk.Label(add, text="Publishing Date", font=("Calibri", 12, "bold"))
    pubdate_label.place(relx=0.15,rely=0.35, anchor="w")
    pubdate_entry=tk.Entry(add, font=("Calibri", 12))
    pubdate_entry.place(relx=0.4, rely=0.35, anchor="w")
    price_label = tk.Label(add, text="Fine Price", font=("Calibri", 12, "bold"))
    price_label.place(relx=0.15,rely=0.40, anchor="w")
    price_entry=tk.Entry(add, font=("Calibri", 12))
    price_entry.place(relx=0.4, rely=0.40, anchor="w")
    quantity_label = tk.Label(add, text="Quantity", font=("Calibri", 12, "bold"))
    quantity_label.place(relx=0.15,rely=0.45, anchor="w")
    quantity_entry=tk.Entry(add, font=("Calibri", 12))
    quantity_entry.place(relx=0.4, rely=0.45, anchor="w")
    add_button = tk.Button(add, text="Enter", command=lambda: addin(isbn_entry, title_entry, author_entry, category_entry, publisher_entry, pubdate_entry, price_entry, quantity_entry), font=("Calibri", 20, "bold"), bg="yellow")
    add_button.place(relx=0.40, rely=0.58, height=50, width=100, anchor="center")
    

    add.mainloop()
def log_out(dashboard):
    dashboard.destroy()
    
    
def open_dashboard(username, st_date, st_time):
    dashboard=tk.Tk()
    dashboard.title("Dashboard")
    dashboard.state("zoomed")
    
    # Create a label to display the admin name
    admin_name_display = tk.Label(dashboard, text="Welcome " + username.title(), font=("Arial", 24, "bold"), fg="green")
    admin_name_display.place(relx=0.02, rely=0.02, anchor="nw")

    # Create a label to display the date and time
    date_label = tk.Label(dashboard, text=st_date, font=("Arial", 12, "bold"))
    date_label.place(relx=0.95, rely=0.02, anchor="ne")

    time_label=tk.Label(dashboard, text=st_time, font=("Arial", 12, "bold"))
    time_label.place(relx=0.95, rely=0.06, anchor="ne")

    # Create the buttons
    add_button = tk.Button(dashboard, text='Add Books', command=add_book, font=("Calibri", 16, "bold"), bg="blue", fg="white")
    add_button.place(relx=0.02, rely=0.30, width=120, height=120, anchor="w")
    issue_button = tk.Button(dashboard, text='Issue Books', command=issue_book, font=("Calibri", 16, "bold"), bg="blue", fg="white")
    issue_button.place(relx=0.2, rely=0.30, width=120, height=120, anchor="w")
    return_button = tk.Button(dashboard, text='Return Books', command=return_book, font=("Calibri", 16, "bold"), bg="blue", fg="white")
    return_button.place(relx=0.38, rely=0.30, width=120, height=120, anchor="w")
    delete_button = tk.Button(dashboard, text='Delete Books', command=delete_book, font=("Calibri", 16, "bold"), bg="blue", fg="white")
    delete_button.place(relx=0.02, rely=0.60, width=120, height=120, anchor="w")
    delete_button = tk.Button(dashboard, text='Delete Books', command=delete_book, font=("Calibri", 16, "bold"), bg="blue", fg="white")
    delete_button.place(relx=0.2, rely=0.60, width=120, height=120, anchor="w")
    search_button = tk.Button(dashboard, text='Search Books', font=("Calibri", 16, "bold"), bg="blue", fg="white")
    search_button.place(relx=0.38, rely=0.60, width=120, height=120, anchor="w")
    show_button = tk.Button(dashboard, text='Show Books', command=show_books, font=("Calibri", 16, "bold"), bg="blue", fg="white")
    show_button.place(relx=0.56, rely=0.45, width=120, height=120, anchor="w")
    lo_button = tk.Button(dashboard, text="Logout", command=lambda:log_out(dashboard), font=("Calibri", 12, "bold"), bg="yellow")
    lo_button.place(relx=0.9, rely=0.85, anchor="e")

    dashboard.mainloop()
    
